# Spotify API

## Authentication and Authorization Tokens

Load client credentials from .env file.

In [2]:
import subprocess
import sys

subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'python-dotenv'])

  Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)


0

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')

Use client credentials to obtain access token for Spotify API. 

See https://developer.spotify.com/documentation/web-api/tutorials/client-credentials-flow for details.

In [4]:
import requests
import time
import base64
import json
import os

TOKEN_ENDPOINT = 'https://accounts.spotify.com/api/token'
TOKEN_FILE = 'spotify_token.json'

class SpotifyAuthenticator:
    def __init__(self):
        self.access_token = None
        self.expires_at = 0
        self.load_token()
        
    def load_token(self):
        """Loads most recent access token."""
        if os.path.exists(TOKEN_FILE):
            with open(TOKEN_FILE, 'r') as file:
                data = json.load(file)
                self.access_token = data.get('access_token')
                self.expires_at = data.get('expires_at', 0)
        
    def save_token(self):
        """For access token persistence."""
        with open(TOKEN_FILE, 'w') as file:
            json.dump({'access_token': self.access_token, 'expires_at': self.expires_at}, file)
        
    def get_token(self):
        """Returns access token or refreshes token if expired."""
        if not self.access_token or time.time() >= self.expires_at:
            self.refresh_token()
        return self.access_token
    
    def refresh_token(self):
        """Requests access token."""
        print('Refreshing access token...')
        
        credentials = f'{CLIENT_ID}:{CLIENT_SECRET}'
        
        # converts credentials: string -> bytes, bytes encode to Base64, bytes -> string
        encoded_credentials = base64.b64encode(credentials.encode()).decode()
        
        headers = {
            'Authorization': f'Basic {encoded_credentials}',
            'Content-Type': 'application/x-www-form-urlencoded'
        }
        data = {'grant_type': 'client_credentials'}
        
        response = requests.post(TOKEN_ENDPOINT, headers=headers, data=data)
        
        if response.status_code == 200:
            token_data = response.json()
            self.access_token = token_data['access_token']
            self.expires_at = time.time() + token_data['expires_in'] - 60
            self.save_token()
            print('Access token refreshed successfully!')
        else:
            raise Exception(f'Could not refresh access token: {response.status_code}, {response.text}')
            
spotify_authenticator = SpotifyAuthenticator()

## Querying by ISRC

In [75]:
import time
import json
import requests

def spotify_search_isrc(isrc_list, category='track', limit=1):
    """Search Spotify for media using ISRC (International Standard Recording Code)."""
    if isinstance(isrc_list, str):
        isrc_list = [isrc_list]

    token = spotify_authenticator.get_token()
    url = 'https://api.spotify.com/v1/search'
    headers = {'Authorization': f'Bearer {token}'}
    
    results = {}
    
    complete = 0
    for index, isrc in enumerate(isrc_list):
        query = f'isrc:{isrc}'
        params = {'q': query, 'type': category, 'limit': limit}

        response = requests.get(url, headers=headers, params=params)

        if response.status_code == 200:
            results[isrc] = response.json()
        else:
            print(f'Error {response.status_code} for ISRC {isrc}: {response.text}')
            results[isrc] = None  # Store None for failed requests

        # adhere to Spotify API rate limit (~180 requests/min -> 2 requests/sec to be safe)
        bundle = 2
        if (index + 1) % bundle == 0:
            complete += bundle
            print(f'Sleeping for 1 second... {complete}/{len(isrc_list)} complete')
            time.sleep(1)

    return results
        
isrc_list = ['QZ7XS2400012', 'NGA3B2214004', 'RUA1H2406425', 'GBAHS1600463', 'MX1722301696']
    
results = spotify_search_isrc(isrc_list)
results_string = json.dumps(result, indent=2)
print(results_string)

Sleeping for 1 second... 2/5 complete
Sleeping for 1 second... 4/5 complete
{
  "QZ7XS2400012": {
    "tracks": {
      "href": "https://api.spotify.com/v1/search?offset=0&limit=1&query=isrc%3AQZ7XS2400012&type=track",
      "limit": 1,
      "next": "https://api.spotify.com/v1/search?offset=1&limit=1&query=isrc%3AQZ7XS2400012&type=track",
      "offset": 0,
      "previous": null,
      "total": 6,
      "items": [
        {
          "album": {
            "album_type": "album",
            "artists": [
              {
                "external_urls": {
                  "spotify": "https://open.spotify.com/artist/4xPQFgDA5M2xa0ZGo5iIsv"
                },
                "href": "https://api.spotify.com/v1/artists/4xPQFgDA5M2xa0ZGo5iIsv",
                "id": "4xPQFgDA5M2xa0ZGo5iIsv",
                "name": "\u00a5$",
                "type": "artist",
                "uri": "spotify:artist:4xPQFgDA5M2xa0ZGo5iIsv"
              },
              {
                "external_urls": {


## Processing

In [100]:
for isrc in isrc_list[:1]:
    base = results[isrc]['tracks']['items']
    
    top_result = base[0]
    
    artist = top_result['artists'][0]['name']
    track = top_result['name']
    album_name = top_result['album']['name']
    
    info = {'Track': track, 'Album Name': album_name, 'Artist': artist}
    
    print(info)

{'Track': 'CARNIVAL', 'Album Name': 'VULTURES 1', 'Artist': '¥$'}
